In [1]:
import pandas as pd
import math

In [19]:
def find_incommings(df_user):
    
    # segment into months
    df_user['year'] = pd.DatetimeIndex(df_user['dates']).year
    df_user['month'] = pd.DatetimeIndex(df_user['dates']).month
    
    # find average income per month
    df_in = df_user[df_user['amount']>0]
    other_account_name = df_in['other_account_name'].unique()
    
    # find average gap in payments
    incomings = {}
    for payment in other_account_name:
        
        df_tmp = df_in[df_in['other_account_name']== payment].copy()
        df_tmp.sort_values(by='dates', inplace=True)
        df_tmp.reset_index(inplace=True, drop=True)
        df_tmp['diff_days'] = pd.to_datetime(df_tmp['dates']).diff().dt.days.fillna(0, downcast='infer')
        
        incomings[payment] = {}
        incomings[payment]['freq'] = math.ceil(df_tmp['diff_days'].mean())
        incomings[payment]['amount'] = df_tmp['amount'].mean()
        incomings[payment]['start_date'] = df_tmp['dates'].min()
           
    return incomings


def find_outgoings(df_user):
    
    # segment into months
    df_user['year'] = pd.DatetimeIndex(df_user['dates']).year
    df_user['month'] = pd.DatetimeIndex(df_user['dates']).month
    
    # find average income per month
    df_in = df_user[df_user['amount']<0]
    other_account_name = df_in['other_account_name'].unique()
        
    # find average gap in payments
    outgoings = {}
    for payment in other_account_name:
        
        df_tmp = df_in[df_in['other_account_name']== payment].copy()
        df_tmp.sort_values(by='dates', inplace=True)
        df_tmp.reset_index(inplace=True, drop=True)
        df_tmp['diff_days'] = pd.to_datetime(df_tmp['dates']).diff().dt.days.fillna(0, downcast='infer')
        
        outgoings[payment] = {}
        outgoings[payment]['freq'] = math.ceil(df_tmp['diff_days'].mean())
        outgoings[payment]['amount'] = df_tmp['amount'].mean()
        outgoings[payment]['start_date'] = df_tmp['dates'].min()
        
        if outgoings[payment]['freq'] > 25:
            outgoings[payment]['Type'] = 'Regular'
        
        if (outgoings[payment]['freq'] >= 7) & (outgoings[payment]['freq'] < 25):
            outgoings[payment]['Type'] = 'Irregular'
            
        if outgoings[payment]['freq'] < 7:
            outgoings[payment]['Type'] = 'Discretionary'
           
    return outgoings


def build_predicted_in_out(user, incommings, outgoings, end_date):
    
    
    in_list = []
    for i in incommings.keys():    
            
        df = pd.DataFrame(data={'dates': pd.date_range(incommings[i]['start_date'], end_date, freq='MS')})
        df['amount_in'] = incommings[i]['amount']
        in_list.append(df)
    
    out_list = []
    for i in outgoings.keys(): 
        
        if outgoings[i]['Type'] == 'Regular':
        
            df = pd.DataFrame(data={'dates': pd.date_range(outgoings[i]['start_date'], end_date, freq=str(outgoings[i]['freq'])+'D')})
            df['amount_out_reg'] = outgoings[i]['amount']
            out_list.append(df)            
        
    df = pd.concat(in_list + out_list)
    df.reset_index(inplace=True,drop=True)
    df.sort_values(by='dates', inplace=True)
    df.reset_index(inplace=True, drop=True)
    df.fillna(0, inplace=True)

    # bucket into months
    df['year'] = pd.DatetimeIndex(df['dates']).year
    df['month'] = pd.DatetimeIndex(df['dates']).month
    
    return df

def gt_buffer(row):
    amount = row['amount']
    buffer = row['buffer']
    txn_type = row['Type']
    other_account_name = row['other_account_name']
    
    alert_flag = False
    if txn_type != 'Regular':
        if abs(amount) > abs(buffer):
            #print('Payment of {} to {} exceeds the projected buffer between regular incoming and outgoing payments this month - are you sure?')
            #alert_dict.update({other_account_name, amount, buffer})
            alert_flag = True
            
    return alert_flag


In [33]:
def forecasting(df):
    # Will be called on a per user basis
    incommings_payments = find_incommings(df.copy())
    outgoing_payments = find_outgoings(df.copy())
    
    # Should be only one user in df
    account_name = list(df['account_name'].value_counts().index)[0]
    
    df_tmp = build_predicted_in_out(account_name, incommings_payments, outgoing_payments, '2024-01-01 00:00:00')
    
    gf_payment_pattern_reg = df_tmp.groupby(['year','month'])[['amount_in','amount_out_reg']].sum()
    gf_payment_pattern_reg['buffer'] = gf_payment_pattern_reg['amount_in'] + gf_payment_pattern_reg['amount_out_reg']
    
    # Make Year and Month columns in df, for use as join keys
    df['year'] = df['dates'].apply(lambda x: int(x[:4]))
    df['month'] = df['dates'].apply(lambda x: int(x[5:7]))
    
    # Promote multi-index
    gf_payment_pattern_reg = gf_payment_pattern_reg.reset_index()
    
    df_join = pd.merge(df,gf_payment_pattern_reg, how = 'left', on = ['year','month'])
    
    df_join['buffer_alert'] = df_join.apply(gt_buffer, axis = 1)
    print(df_join.head())
    print(df_join.to_json())
    
    df_json = df_join.to_json()
        
    return df_json
    
    
    
    
    
    
    
    
    
    
    
    
    

In [4]:
df = pd.read_csv('gs://shakingshamrocks_eu/test_data_3_sec.csv')

In [5]:
df = df.drop(df.columns[0],axis = 1)

In [7]:
df = df.loc[df['account_name'] == 'Katherine Valencia']

In [8]:
df.head()

,dates,account_name,account_IBAN,other_account_name,Type,amount,balance
0,2018-02-16 02:17:13,Katherine Valencia,DOJO46309747392475,Wages MyOrg,Regular,2300.000000,2300.000000
2,2018-02-25 21:12:59,Katherine Valencia,DOJO46309747392475,Food Reg,Irregular,-220.577729,4679.422271
3,2018-02-28 01:18:39,Katherine Valencia,DOJO46309747392475,Subscription,Regular,-55.000000,4624.422271
4,2018-02-28 03:02:54,Katherine Valencia,DOJO46309747392475,Phone,Regular,-18.000000,4606.422271
7,2018-02-28 07:15:31,Katherine Valencia,DOJO46309747392475,Car,Regular,-330.000000,2572.323012


In [34]:
forecasting(df)

                 dates        account_name        account_IBAN  \
0  2018-02-16 02:17:13  Katherine Valencia  DOJO46309747392475   
1  2018-02-25 21:12:59  Katherine Valencia  DOJO46309747392475   
2  2018-02-28 01:18:39  Katherine Valencia  DOJO46309747392475   
3  2018-02-28 03:02:54  Katherine Valencia  DOJO46309747392475   
4  2018-02-28 07:15:31  Katherine Valencia  DOJO46309747392475   

  other_account_name       Type       amount      balance  year  month  \
0        Wages MyOrg    Regular  2300.000000  2300.000000  2018      2   
1           Food Reg  Irregular  -220.577729  4679.422271  2018      2   
2       Subscription    Regular   -55.000000  4624.422271  2018      2   
3              Phone    Regular   -18.000000  4606.422271  2018      2   
4                Car    Regular  -330.000000  2572.323012  2018      2   

   amount_in  amount_out_reg  buffer  buffer_alert  
0        0.0         -1883.0 -1883.0         False  
1        0.0         -1883.0 -1883.0         False  

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


'{"dates":{"0":"2018-02-16 02:17:13","1":"2018-02-25 21:12:59","2":"2018-02-28 01:18:39","3":"2018-02-28 03:02:54","4":"2018-02-28 07:15:31","5":"2018-02-28 08:09:57","6":"2018-02-28 08:19:09","7":"2018-02-28 08:43:13","8":"2018-02-28 08:49:09","9":"2018-03-01 01:32:13","10":"2018-03-05 02:51:29","11":"2018-03-07 13:23:32","12":"2018-03-11 03:45:17","13":"2018-03-14 00:41:47","14":"2018-03-15 17:13:08","15":"2018-03-19 05:03:24","16":"2018-03-21 16:05:40","17":"2018-03-25 11:01:36","18":"2018-03-28 00:27:57","19":"2018-03-28 05:40:32","20":"2018-03-28 07:16:06","21":"2018-03-28 08:38:13","22":"2018-03-28 12:57:06","23":"2018-03-28 12:59:38","24":"2018-03-28 14:55:52","25":"2018-04-01 06:33:20","26":"2018-04-04 00:26:55","27":"2018-04-04 05:13:30","28":"2018-04-08 22:57:47","29":"2018-04-11 00:11:26","30":"2018-04-12 03:49:42","31":"2018-04-15 04:18:51","32":"2018-04-16 02:25:01","33":"2018-04-18 00:15:00","34":"2018-04-19 05:25:45","35":"2018-04-22 14:30:09","36":"2018-04-25 00:17:12",